In [19]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import pandas as pd
from keras.layers import Dropout

In [33]:
# packages utiles
import numpy as np
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
import sklearn.metrics
from sklearn.linear_model import LinearRegression
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from sklearn.linear_model import lasso_path
import seaborn as sns

In [34]:
data = pd.read_csv('Données élections et INSEE.csv')
data.drop(columns=['Unnamed: 0'], inplace=True)
data = data.loc[data['Tentative de réélection']==True]

/var/folders/99/gq6w32xj1_b7n4b18rr2czgr0000gn/T/ipykernel_5671/2837926579.py:1: DtypeWarning: Columns (1,1911) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('Données élections et INSEE.csv')


In [35]:
df2 = data.select_dtypes(include=np.number)
df2.replace([np.inf, -np.inf], np.nan, inplace=True)
df2 = df2.fillna(0)
col_names = df2.loc[:, df2.columns != 'per_gop'].columns.values
features = df2[col_names]
features = StandardScaler().fit(features.values).transform(features.values)
df2[col_names] = features
#df2.head()

In [39]:
#3. Estimer un modèle LASSO et afficher les valeurs des coefficients
lasso1 = Lasso(fit_intercept=True, alpha = 0.1).fit(X_train,y_train)

#np.abs(lasso1.coef_)
features_selec = df2.select_dtypes(include=np.number).columns[np.abs(lasso1.coef_)>0].tolist()
features_selec
# Le modèle est assez parcimonieux puisque 
# un sous-échantillon de variables sont sélectionnées 

ValueError: could not convert string to float: 's'

In [36]:
data['y'] = data.apply( lambda row: row['Nom de l\'élu en 2014'] == row['Nom de l\'élu en 2020'], axis=1)
data['y'] = data['y'].apply(lambda row: int(row))
y = data['y']

cols = ['CODGEO', 'Nom de l\'élu en 2020', 'Nom de l\'élu en 2014', 'Libellé de la commune',
       'Tentative de réélection', 'y']
cols2 = list(data.columns)
for i in cols:
    cols2.remove(i)
    
X = data[cols2]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [4]:
model = Sequential()
model.add(Dense(12, input_shape=(8,), activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [5]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [6]:
model.fit(X, y, epochs=150, batch_size=10)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type float).

In [32]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Define the parameters to optimize
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30],
    'units_1': [16, 32, 64],
    'units_2': [8, 16, 32],
    'dropout': [0.2, 0.4, 0.6]
}

# Define the model
def create_model(optimizer, batch_size, epochs, units_1, units_2, dropout):
    model = Sequential()
    model.add(Dense(units=units_1, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(units=units_2, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Use RandomizedSearchCV to find the best combination of parameters
search = RandomizedSearchCV(estimator=create_model, param_distributions=param_grid, n_iter=10, cv=3, scoring='accuracy', verbose=1)

# Train the model using the best combination of parameters
if len(X) > 10000:
    search.fit(X_train, y_train, batch_size=128, epochs=30)
else:
    search.fit(X_train, y_train)

# Make predictions on the test data
y_pred = search.predict(X_test)


TypeError: estimator should be an estimator implementing 'fit' method, <function create_model at 0x2870d1510> was passed

In [33]:
from keras.models import clone_model

# Define the parameters to optimize
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'batch_size': [32, 64, 128],
    'epochs': [10, 20, 30],
    'units_1': [16, 32, 64],
    'units_2': [8, 16, 32],
    'dropout': [0.2, 0.4, 0.6]
}

# Define the model
def create_model(optimizer, batch_size, epochs, units_1, units_2, dropout):
    model = Sequential()
    model.add(Dense(units=units_1, activation='relu', input_dim=X.shape[1]))
    model.add(Dense(units=units_2, activation='relu'))
    model.add(Dropout(dropout))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Create an instance of the Keras model
model = create_model(optimizer='adam', batch_size=32, epochs=10, units_1=16, units_2=8, dropout=0.2)

# Use the clone_model() function to create a clone of the model
model_clone = clone_model(model)

# Use RandomizedSearchCV to find the best combination of parameters
search = RandomizedSearchCV(estimator=model_clone, param_distributions=param_grid, n_iter=10, cv=3, scoring='accuracy', verbose=1)

# Train the model using the best combination of parameters
if len(X) > 10000:
    search.fit(X_train, y_train, batch_size=128, epochs=30)
else:
    search.fit(X_train, y_train)

# Make predictions on the test data
y_pred = search.predict(X_test)


TypeError: Cannot clone object '<keras.engine.sequential.Sequential object at 0x2871e8850>' (type <class 'keras.engine.sequential.Sequential'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' method.